[![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-181717?style=for-the-badge&logo=GitHub&link=https://github.com/Mearman/openalex-docs)](https://github.com/Mearman/openalex-docs)[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-181717?style=for-the-badge&logo=github&link=https://github.com/Mearman/openalex-docs/blob/main/quickstart-tutorial.ipynb)](https://github.com/Mearman/openalex-docs/blob/main/quickstart-tutorial.ipynb)[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=for-the-badge&logo=Google%20Colab&link=https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/quickstart-tutorial.ipynb)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/quickstart-tutorial.ipynb)

In [25]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python-pydantic-v1.git"
%pip install pandasai

  Cloning https://github.com/Mearman/openalex-python-pydantic-v1.git to /private/var/folders/c5/v5yq8fds13j2gh_stx9crvtm0000gn/T/pip-req-build-qk1bjvhc
  Running command git clone --filter=blob:none --quiet https://github.com/Mearman/openalex-python-pydantic-v1.git /private/var/folders/c5/v5yq8fds13j2gh_stx9crvtm0000gn/T/pip-req-build-qk1bjvhc
  Resolved https://github.com/Mearman/openalex-python-pydantic-v1.git to commit 375613c3f4c2ca33cc2686b51347a57d2ceee17b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pydantic-1.10.14-py3-none-any.whl.metadata (150 kB)
Using cached pydantic-1.10.14-py3-none-any.whl (158 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 2.0 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.3
    Uninstalling pydantic-2.5.3:
      Successfully uninstalled

In [26]:
import json
import pandas as pd
import numpy as np
from openalex_api import Configuration, ApiClient,AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

ImportError: cannot import name 'validate_call' from 'pydantic' (/Users/joe/.asdf/installs/python/3.12.1/lib/python3.12/site-packages/pydantic/__init__.py)

In [ ]:
from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
openapi_token = "" # @param {type:"string"}

# Quickstart tutorial

Lets use the OpenAlex API to get journal articles and books published by authors at Stanford University. We'll limit our search to articles published between 2010 and 2020. Since OpenAlex is free and openly available, these examples work without any login or account creation. :thumbsup:

{% hint style="info" %}
If you open these examples in a web browser, they will look _much_ better if you have a browser plug-in such as [JSONVue](https://chrome.google.com/webstore/detail/jsonvue/chklaanhfefbnpoihckbnefhakgolnmc) installed.
{% endhint %}

### 1. Find the institution&#x20;

You can use the [institutions](./api-entities/institutions/README.ipynb) endpoint to learn about universities and research centers. OpenAlex has a powerful search feature that searches across 108,000 institutions.

Lets use it to search for Stanford University:

* Find Stanford University\
  [`https://api.openalex.org/institutions?search=stanford`](https://api.openalex.org/institutions?search=stanford)

In [ ]:
response = institutions_api.get_institutions(
	search="stanford"
)

display(pd.DataFrame(response.results))

In [ ]:
try:
	print(openapi_token)
except:
	raise Exception("Please provide an openapi_token")

df = pd.DataFrame(response.results)
numeric_df = df.select_dtypes(include=[np.number])
display(numeric_df)

llm = OpenAI(api_token=openapi_token)
sdf = SmartDataframe(numeric_df, config={"llm": llm})
sdf.chat("Plot a chart of this data")

Our first result looks correct (yeah!):

```json
{
  "id": "https://openalex.org/I97018004",
  "ror": "https://ror.org/00f54p054",
  "display_name": "Stanford University",
  "country_code": "US",
  "type": "education",
  "homepage_url": "http://www.stanford.edu/"
  // other fields removed
}
```

We can use the ID `https://openalex.org/I97018004` in that result to find out more.

### &#x20;2. Find articles (works) associated with Stanford University

The [works](./api-entities/works/README.ipynb) endpoint contains over 240 million articles, books, and theses :astonished:. We can filter to show works associated with Stanford.

* Show works where at least one author is associated with Stanford University\
  [`https://api.openalex.org/works?filter=institutions.id:https://openalex.org/I97018004`](https://api.openalex.org/works?filter=institutions.id:https://openalex.org/I97018004)

In [ ]:
response = works_api.get_works(
	filter="institutions.id:https://openalex.org/I97018004"
)

display(pd.DataFrame(response.results))

In [ ]:
try:
	print(openapi_token)
except:
	raise Exception("Please provide an openapi_token")

df = pd.DataFrame(response.results)
numeric_df = df.select_dtypes(include=[np.number])
display(numeric_df)

llm = OpenAI(api_token=openapi_token)
sdf = SmartDataframe(numeric_df, config={"llm": llm})
sdf.chat("Plot a chart of this data")

This is just one of the 50+ ways that you can filter works!

### 3. Filter works by publication year

Right now the list shows records for all years. Lets narrow it down to works that were published between 2010 to 2020, and sort from newest to oldest.

* Show works with publication years 2010 to 2020, associated with Stanford University\
  [https://api.openalex.org/works?filter=institutions.id:https://openalex.org/I97018004,publication\_year:2010-2020\&sort=publication\_date:desc](https://api.openalex.org/works?filter=institutions.id:https://openalex.org/I97018004,publication\_year:2010-2020\&sort=publication\_date:desc)

In [ ]:
response = works_api.get_works(
	filter="institutions.id:https://openalex.org/I97018004,publication_year:2010-2020",
	sort="publication_date:desc"
)

display(pd.DataFrame(response.results))

In [ ]:
try:
	print(openapi_token)
except:
	raise Exception("Please provide an openapi_token")

df = pd.DataFrame(response.results)
numeric_df = df.select_dtypes(include=[np.number])
display(numeric_df)

llm = OpenAI(api_token=openapi_token)
sdf = SmartDataframe(numeric_df, config={"llm": llm})
sdf.chat("Plot a chart of this data")

### 4. Group works by publication year to show counts by year

Finally, you can group our result by publication year to get our final result, which is the number of articles produced by Stanford, by year from 2010 to 2020. There are more than 30 ways to group records in OpenAlex, including by publisher, journal, and open access status.

* Group records by publication year\
  [`https://api.openalex.org/works?filter=institutions.id:https://openalex.org/I97018004,publication\_year:2010-2020\&group-by=publication\_year`](https://api.openalex.org/works?filter=institutions.id:https://openalex.org/I97018004,publication\_year:2010-2020\&group-by=publication\_year)

In [ ]:
response = works_api.get_works(
	filter="institutions.id:https://openalex.org/I97018004,publication_year:2010-2020",
	group_by="publication_year"
)

display(pd.DataFrame(response.results))

In [ ]:
try:
	print(openapi_token)
except:
	raise Exception("Please provide an openapi_token")

df = pd.DataFrame(response.results)
numeric_df = df.select_dtypes(include=[np.number])
display(numeric_df)

llm = OpenAI(api_token=openapi_token)
sdf = SmartDataframe(numeric_df, config={"llm": llm})
sdf.chat("Plot a chart of this data")

That gives a result like this:

```json
[
  {
    "key": "2020",
    "key_display_name": "2020",
    "count": 18627
  },
  {
    "key": "2019",
    "key_display_name": "2019",
    "count": 15933
  },
  {
    "key": "2017",
    "key_display_name": "2017",
    "count": 14789
  },
  ...
]
```

There you have it! This same technique can be applied to hundreds of questions around scholarly data. The data you received is under a [CC0 license](https://creativecommons.org/publicdomain/zero/1.0/), so not only did you access it easily, you can share it freely! :tada:

## What's next?

Jump into an area of OpenAlex that interests you:

* [Works](./api-entities/works/README.ipynb)
* [Authors](./api-entities/authors/README.ipynb)
* [Sources](./api-entities/sources/README.ipynb)
* [Institutions](./api-entities/institutions/README.ipynb)
* [Concepts](./api-entities/concepts/README.ipynb)
* [Publishers](./api-entities/publishers/README.ipynb)
* [Funders](./api-entities/funders/README.ipynb)

And check out our [tutorials](./additional-help/tutorials.md) page for some hands-on examples!